# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [2]:
! pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark 
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName('HW').getOrCreate()
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [4]:
data = spark.read.csv('pga_tour_historical.csv', inferSchema = True, header = True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [5]:
data.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [6]:
data.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [7]:
from pyspark.sql.types import *

In [8]:
data_schema = [StructField('name', StringType(), True), 
               StructField('Season', IntegerType(), True), 
               StructField('Statistic', StringType(), True), 
               StructField('Variable', StringType(), True), 
               StructField('Value', IntegerType(), True)]

In [9]:
final_struct = StructType(fields = data_schema)

In [11]:
history = spark.read.csv('pga_tour_historical.csv', schema = final_struct, header = True)

In [12]:
history.limit(5).toPandas()

,name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


In [13]:
history.printSchema()

root
 |-- name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [16]:
history.select('Value').describe().show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|            462538|
|   mean|14470.056730474036|
| stddev|169211.89008189933|
|    min|              -152|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [17]:
history.select(['Value', 'Season']).summary('count', 'mean', 'stddev', 'min', 'max').show()

+-------+------------------+------------------+
|summary|             Value|            Season|
+-------+------------------+------------------+
|  count|            462538|            757065|
|   mean|14470.056730474036|2010.7434658846994|
| stddev|169211.89008189933|0.7288150612632263|
|    min|              -152|              2010|
|    max|           3564954|              2012|
+-------+------------------+------------------+



## We're done! Great job!